In [30]:
pip install jqdatasdk

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from datetime import datetime, timedelta
import jqdatasdk as jq
import os

In [3]:
from dotenv import load_dotenv
load_dotenv() 
acc = os.getenv('ACCOUNT')
pas = os.getenv('PASSWORD')

In [4]:

class FuturesInfo:
    data_exist = []
    empty = []

    def __init__(self, f_start, f_end, f_kind, the_path):
        self.f_start = f_start
        self.f_end = f_end
        self.f_kind = f_kind
        self.path = the_path

    def get_large_data(self):
        """
        Get the futures data and download the datasets to local

        Parameters
        ----------
        f_start : time string, the time you start collecting the data
          A string
        f_end : time string, the time you end collecting the data
          A string
        f_kind: the substring of the futures ticker. Category of futures types
          A string
        the_path: the location you want to download data to
          A string

        Returns
        -------
        print out all the futures data downloaded. Return one of the future's dataframe. Downloaded all specified futures to specified location

        Examples
        --------
        >>> from magic_futures_data_acquisition_pkg import magic_futures_data_acquisition_pk
        >>> get_large = FuturesInfo('2020-01-01','2020-03-01','AP','/Users/tylerwang/Desktop/get_futures_data/')
        >>> get_large.get_large_data()
        ['', 'AP1805.XZCE', 'AP1807.XZCE', 'AP1810.XZCE', 'AP1811.XZCE', 'AP1812.XZCE', 'AP1901.XZCE', 'AP1903.XZCE', 'AP1905.XZCE', 'AP1907.XZCE', 'AP1910.XZCE', 'AP1911.XZCE', 'AP1912.XZCE', 'AP2001.XZCE', 'AP2003.XZCE', 'AP2005.XZCE', 'AP2007.XZCE', 'AP2010.XZCE', 'AP2011.XZCE', 'AP2012.XZCE', 'AP2101.XZCE', 'AP2103.XZCE', 'AP2105.XZCE', 'AP2110.XZCE', 'AP2111.XZCE', 'AP2112.XZCE', 'AP2201.XZCE', 'AP2203.XZCE', 'AP2204.XZCE', 'AP2205.XZCE', 'AP2210.XZCE', 'AP2211.XZCE', 'AP2212.XZCE', 'AP2301.XZCE', 'AP2303.XZCE', 'AP2304.XZCE', 'AP2305.XZCE', 'AP2310.XZCE', 'AP2311.XZCE', 'AP8888.XZCE', 'AP9999.XZCE']
        	time	open	close	high	low	volume	money
        0	2020-01-02	7740.0	7741.0	7791.0	7706.0	93362.0	7.237311e+09
        1	2020-01-03	7736.0	7589.0	7767.0	7580.0	149249.0	1.148803e+10
        2	2020-01-06	7560.0	7474.0	7577.0	7428.0	168525.0	1.261648e+10
        3	2020-01-07	7474.0	7501.0	7528.0	7443.0	107722.0	8.070493e+09
        4	2020-01-08	7505.0	7497.0	7556.0	7424.0	128109.0	9.595661e+09
        5	2020-01-09	7497.0	7360.0	7545.0	7358.0	145093.0	1.083167e+10
        """
        global data_exist
        global empty
        f = open("futures_list.txt", "w+")
        lis = f.read()
        data_exist = lis.split(" ")
        # log in the  JQ account
        jq.auth(acc, pas)
        all_futures = jq.get_all_securities(['futures'])
        # name the index of the DataFrame
        all_futures.index.name = 'id'
        # convert index column to list
        futures_idx = all_futures.index.values.tolist()
        substring = self.f_kind
        empty = []
        for idx in futures_idx:
            if substring in idx:
                empty.append(idx)
        count = 0
        for item in empty:
            future_info_yearly = jq.get_price(security=item, start_date=datetime.strptime(self.f_start, '%Y-%m-%d'),
                                              end_date=datetime.strptime(self.f_end, '%Y-%m-%d'), frequency='1d')
            str = item + '.csv'
            future_info_yearly.index.name = 'time'
            future_info_yearly.dropna(inplace=True)
            future_info_yearly.to_csv(os.path.join(self.path, str), mode='a')
            if item not in data_exist:
                data_exist.append(item)

            count += 1
            print(count)
        f = open("futures_list.txt", "w")
        f.write(" ".join(empty))
        f.close()
        for item in data_exist:
            if item == "":
                continue
            str = item + '.csv'
            df = pd.read_csv(self.path + str)
            df.drop_duplicates(subset=['time'], inplace=True)
            df.sort_values(by='time', inplace=True)
            df = df[df.time.str.contains('time') == False]
            df.to_csv(os.path.join(self.path, str), index=False, index_label='time')
        print(data_exist)
        return df


    def get_missing_data(self):
        """
        Find the missing date in existing datasets and download the missing data to local

        Parameters
        ----------
        f_start : time string, the time you start collecting the data
          A string
        f_end : time string, the time you end collecting the data
          A string
        f_kind: the substring of the futures ticker. Category of futures types
          A string
        the_path: the location you want to download data to
          A string

        Returns
        -------
        print out the missing date for all existing futures and search for the missing date in API and download the datasets to local
        return the new dataframe contains missing data
        Examples
        --------
        >>> from magic_futures_data_acquisition_pkg import magic_futures_data_acquisition_pk
        >>> get_miss = FuturesInfo('2020-01-01', '2020-02-01', 'AP','/Users/tylerwang/Desktop/get_futures_data/')
        >>> get_miss.get_missing_data()
        AP1805.XZCE:['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31']
        	time	open	close	high	low	volume	money
        0	2020-01-02	7740.0	7741.0	7791.0	7706.0	93362.0	7.237311e+09
        1	2020-01-03	7736.0	7589.0	7767.0	7580.0	149249.0	1.148803e+10
        2	2020-01-06	7560.0	7474.0	7577.0	7428.0	168525.0	1.261648e+10
        3	2020-01-07	7474.0	7501.0	7528.0	7443.0	107722.0	8.070493e+09
        4	2020-01-08	7505.0	7497.0	7556.0	7424.0	128109.0	9.595661e+09
        5	2020-01-09	7497.0	7360.0	7545.0	7358.0	145093.0	1.083167e+10
        """
        global data_exist
        global path
        f = open("futures_list.txt", "r+")
        lis = f.read()
        data_exist = lis.split(" ")
        futures_dict = {}
        substring = self.f_kind
        for item in data_exist:
            if item == "":
                continue
            if substring in item:
                str = item + '.csv'
                future_info = pd.read_csv(self.path + str)
                # groupby datetime and get the size of futures of that day
                date_with_data = future_info.groupby(['time']).size()
                all_date = pd.date_range(start=datetime.strptime(self.f_start, '%Y-%m-%d'),
                                     end=datetime.strptime(self.f_end, '%Y-%m-%d'))
                date_with_data.index = pd.DatetimeIndex(date_with_data.index)
                # set new index to data series, if its NaN then 0
                date_with_data = date_with_data.reindex(all_date, fill_value=0)
                # get the date of missing data
                date_with_data = date_with_data[date_with_data.index.dayofweek < 5]
                miss_date_list = []
                for index, value in date_with_data.items():
                    if value == 0:
                        miss_date_list.append(index.strftime('%Y-%m-%d'))
                        futures_dict[item] = miss_date_list
        for key, value in futures_dict.items():
            print(key, end=":")
            print(value)
        jq.auth(acc, pas)
        count = 0
        for item in futures_dict:
            date_list = []
            for time in futures_dict[item]:
                date_list.append(datetime.strptime(time, '%Y-%m-%d'))
            max_date = max(date_list)
            min_date = min(date_list)
            future_info = jq.get_price(security=item,
                                           start_date=min_date,
                                           end_date=max_date, frequency='1d')
            str = item + '.csv'
            future_info.to_csv(os.path.join(self.path, str), mode='a')
            count += 1
            print(count)
        for item in data_exist:
            if item == "":
                continue
            str = item + '.csv'
            df = pd.read_csv(self.path + str)
            df.drop_duplicates(subset=['time'], inplace=True)
            df.sort_values(by='time', inplace=True)
            df.dropna(inplace = True)
            df.to_csv(os.path.join(self.path, str), index=False, index_label='time')
        return df

In [11]:
#This is used for initial data acquisition, as you can see it records what tickers have been downloaded to local and one of the dataframe created. 
#In local, we can see all futures with substring 'AP' downloaded. There are 40 futures with substring 'AP'.
get_large = FuturesInfo('2020-01-01','2020-02-01','AP','/Users/tylerwang/Desktop/get_futures_data/')
get_large.get_large_data()[:5]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
['', 'AP1805.XZCE', 'AP1807.XZCE', 'AP1810.XZCE', 'AP1811.XZCE', 'AP1812.XZCE', 'AP1901.XZCE', 'AP1903.XZCE', 'AP1905.XZCE', 'AP1907.XZCE', 'AP1910.XZCE', 'AP1911.XZCE', 'AP1912.XZCE', 'AP2001.XZCE', 'AP2003.XZCE', 'AP2005.XZCE', 'AP2007.XZCE', 'AP2010.XZCE', 'AP2011.XZCE', 'AP2012.XZCE', 'AP2101.XZCE', 'AP2103.XZCE', 'AP2105.XZCE', 'AP2110.XZCE', 'AP2111.XZCE', 'AP2112.XZCE', 'AP2201.XZCE', 'AP2203.XZCE', 'AP2204.XZCE', 'AP2205.XZCE', 'AP2210.XZCE', 'AP2211.XZCE', 'AP2212.XZCE', 'AP2301.XZCE', 'AP2303.XZCE', 'AP2304.XZCE', 'AP2305.XZCE', 'AP2310.XZCE', 'AP2311.XZCE', 'AP8888.XZCE', 'AP9999.XZCE']


In [8]:
#Here we want to downloaded one more month for the futures 'AP' data. We can see all tickers which have been downloaded to local.
#The program will ignore the datasets already downloaded to local and append more data.
get_large = FuturesInfo('2020-01-01','2020-03-01','AP','/Users/tylerwang/Desktop/get_futures_data/')
get_large.get_large_data()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
['', 'AP1805.XZCE', 'AP1807.XZCE', 'AP1810.XZCE', 'AP1811.XZCE', 'AP1812.XZCE', 'AP1901.XZCE', 'AP1903.XZCE', 'AP1905.XZCE', 'AP1907.XZCE', 'AP1910.XZCE', 'AP1911.XZCE', 'AP1912.XZCE', 'AP2001.XZCE', 'AP2003.XZCE', 'AP2005.XZCE', 'AP2007.XZCE', 'AP2010.XZCE', 'AP2011.XZCE', 'AP2012.XZCE', 'AP2101.XZCE', 'AP2103.XZCE', 'AP2105.XZCE', 'AP2110.XZCE', 'AP2111.XZCE', 'AP2112.XZCE', 'AP2201.XZCE', 'AP2203.XZCE', 'AP2204.XZCE', 'AP2205.XZCE', 'AP2210.XZCE', 'AP2211.XZCE', 'AP2212.XZCE', 'AP2301.XZCE', 'AP2303.XZCE', 'AP2304.XZCE', 'AP2305.XZCE', 'AP2310.XZCE', 'AP2311.XZCE', 'AP8888.XZCE', 'AP9999.XZCE']


time    open   close    high     low    volume          money
0   2020-01-02  7740.0  7741.0  7791.0  7706.0   93362.0   7237310800.0
1   2020-01-03  7736.0  7589.0  7767.0  7580.0  149249.0  11488030400.0
2   2020-01-06  7560.0  7474.0  7577.0  7428.0  168525.0  12616475200.0
3   2020-01-07  7474.0  7501.0  7528.0  7443.0  107722.0   8070493200.0
4   2020-01-08  7505.0  7497.0  7556.0  7424.0  128109.0   9595660900.0
5   2020-01-09  7497.0  7360.0  7545.0  7358.0  145093.0  10831665900.0
6   2020-01-10  7360.0  7447.0  7477.0  7333.0  135702.0  10063703200.0
7   2020-01-13  7461.0  7289.0  7469.0  7273.0  144949.0  10622566800.0
8   2020-01-14  7284.0  7302.0  7360.0  7284.0   97103.0   7105570500.0
9   2020-01-15  7300.0  7325.0  7380.0  7242.0  134238.0   9801800400.0
10  2020-01-16  7328.0  7351.0  7396.0  7317.0  106995.0   7878399400.0
11  2020-01-17  7356.0  7469.0  7500.0  7352.0  145179.0  10814477700.0
12  2020-01-20  7422.0  7421.0  7480.0  7377.0   91335.0   6785709300.0
13  2020-01-21  7427.0  7339.0  7485.0  7324.0  118127.0   8769156000.0
14  2020-01-22  7339.0  7361.0  7425.0  7281.0  102766.0   7550640700.0
15  2020-01-23  7369.0  7420.0  7445.0  7331.0  105971.0   7833231300.0
33  2020-02-03  6895.0  6874.0  7079.0  6874.0  186578.0  12975851000.0
34  2020-02-04  6850.0  7002.0  7048.0  6836.0  126561.0   8825565500.0
35  2020-02-05  6995.0  7011.0  7095.0  6981.0  144869.0  10191854000.0
36  2020-02-06  7031.0  7010.0  7046.0  6988.0   97118.0   6811193200.0
37  2020-02-07  7010.0  7026.0  7076.0  6916.0  120079.0   8445843500.0
38  2020-02-10  7001.0  7008.0  7064.0  6981.0  114979.0   8080999400.0
39  2020-02-11  7000.0  7004.0  7032.0  6971.0  110375.0   7731446400.0
40  2020-02-12  7000.0  6990.0  7050.0  6917.0  164659.0  11497016000.0
41  2020-02-13  6990.0  6933.0  7020.0  6910.0  164111.0  11433654800.0
42  2020-02-14  6939.0  6866.0  6950.0  6852.0  174008.0  11990337000.0
43  2020-02-17  6850.0  6993.0  7002.0  6817.0  199200.0  13777861800.0
44  2020-02-18  6998.0  6982.0  7037.0  6960.0  171271.0  11979316100.0
45  2020-02-19  6972.0  6997.0  7045.0  6972.0  132739.0   9298206400.0
46  2020-02-20  6996.0  7252.0  7267.0  6984.0  398197.0  28523104200.0
47  2020-02-21  7245.0  7277.0  7309.0  7211.0  233644.0  16975651600.0
48  2020-02-24  7280.0  7277.0  7358.0  7161.0  235077.0  17133080900.0
49  2020-02-25  7205.0  7283.0  7315.0  7191.0  183889.0  13374292300.0
50  2020-02-26  7305.0  7126.0  7355.0  7120.0  269713.0  19549903700.0
51  2020-02-27  7100.0  6872.0  7133.0  6840.0  388897.0  27023018100.0
52  2020-02-28  6818.0  6763.0  6896.0  6701.0  370099.0  25122715800.0

In [44]:
# If we are not sure if we downloaded datasets in a specified time range.
# We can use this function to check the missing data. The texts show for each ticker on which date, there is no record.
# If there is suppose to have record, then automaticlly update missing records to local. (Except Holidays and weekends.)
get_miss = FuturesInfo('2020-01-01', '2020-02-01', 'AP','/Users/tylerwang/Desktop/get_futures_data/')
get_miss.get_missing_data()

AP1805.XZCE:['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31']
AP1807.XZCE:['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-31']
AP1810.XZCE:['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10', '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17', '2020-01-20', '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-27', '2020-01-28', '2020-01-29', '2020-01-30', '2020-01-

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
